In [ ]:
!sudo apt-get install -y xvfb ffmpeg
!pip install gym
!pip install 'imageio==2.4.0'
!pip install PILLOW
!pip install pyvirtualdisplay
!pip install tf-agents

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 784 kB of archives.
After this operation, 2,270 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.8 [784 kB]
Fetched 784 kB in 1s (861 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected pack

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.trajectories import time_step as ts

from tf_agents.environments import suite_gym
from tf_agents.networks import q_network
from tf_agents.agents.dqn import dqn_agent
from tf_agents.utils import common
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory

from tf_agents.agents.categorical_dqn import categorical_dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import categorical_q_network

tf.compat.v1.enable_v2_behavior()

import random

In [ ]:
from google.colab import drive 

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import numpy as np
from typing import Any, Optional, Text
import six
from tf_agents.typing import types

In [ ]:
class Fighter:
  def __init__(self, _health, _current_loc, name):
    self.health = _health
    self.current_loc = _current_loc
    self.groggy_count = 0 
    self.delay = 0
    self.current_action = 9
    self.previous_action = 9
    self.combo_stack = 0
    self.name = name

    # action spec
    self.normal_punch_distance = 150
    self.normal_punch_dmg = 11
    self.normal_punch_delay = 0
    self.normal_punch_groggy = 0

    self.normal_punch_2_distance = 160
    self.normal_punch_2_dmg = 10
    self.normal_punch_2_delay = 0
    self.normal_punch_2_groggy = 0

    self.normal_kick_distance = 200
    self.normal_kick_dmg = 8
    self.normal_kick_delay = 0 
    self.normal_kick_groggy = 0 

    self.normal_kick_2_distance = 140
    self.normal_kick_2_dmg = 12
    self.normal_kick_2_delay = 0
    self.normal_kick_2_groggy = 0

    self.smash_punch_distance = 170
    self.smash_punch_dmg = 15
    self.smash_punch_delay = 1
    self.smash_punch_groggy = 2

    self.smash_punch_2_distance = 140
    self.smash_punch_2_dmg = 25
    self.smash_punch_2_delay = 2
    self.smash_punch_2_groggy = 3

    self.smash_kick_distance = 270
    self.smash_kick_dmg = 25
    self.smash_kick_delay = 2
    self.smash_kick_groggy = 4 

    self.smash_kick_2_distance = 240
    self.smash_kick_2_dmg = 20
    self.smash_kick_2_delay = 2
    self.smash_kick_2_groggy = 3

    self.smash_kick_3_distance = 170 
    self.smash_kick_3_dmg = 30
    self.smash_kick_3_delay = 2
    self.smash_kick_3_groggy = 4

    self.step = 50

    pass

  def move_left(self): # Action Index 0
    # move는 20씩 이동하되 
    # 왼쪽으로 이동시 current_loc값이 음수가 되는것을 고려
    self.current_action = 0
    _action_idx = 0
    _action_distance = 9999
    _action_dmg = 0
    _delay = 0
    _groggy = 0
    update_loc = self.current_loc - self.step
    if self.groggy_count == 0 and self.delay == 0:
      if update_loc < 0:
        self.current_loc = 0
        print(self.name,"가 좌로 이동 수행 현재위치는 : ", self.current_loc)
      else:
        self.current_loc = update_loc
        print(self.name,"가 좌로 이동 수행 현재위치는 : ", self.current_loc)
      return _action_idx, _action_distance, _action_dmg, _delay, _groggy
    else:
      # 액션 수행 불가
      print(self.name,"가 좌로 이동을 수행하려했으나 현재는 수행불가합니다", self.current_loc)
      return -1, -1, -1, -1, -1

  def move_right(self): # Action Index 1
    # move는 20씩 이동하되 
    # 오른쪽으로 이동시 current_loc값이 1000 이상이 되는것을 고려
    self.current_action = 1
    _action_idx = 1
    _action_distance = 9999
    _action_dmg = 0
    _delay = 0
    _groggy = 0
    update_loc = self.current_loc + self.step
    if self.groggy_count == 0 and self.delay == 0:
      if update_loc > 1000:
        self.current_loc = 1000
        print(self.name,"가 우로 이동 수행 현재위치는 : ", self.current_loc)
      else:
        self.current_loc = update_loc
        print(self.name,"가 우로 이동 수행 현재위치는 : ", self.current_loc)
      return _action_idx, _action_distance, _action_dmg, _delay, _groggy
    else:
      # 액션 수행 불가
      print(self.name,"가 우로 이동을 수행하려했으나 현재는 수행불가합니다", self.current_loc)
      return -1, -1, -1, -1, -1

  def normal_punch(self): # Action Index 2
    self.current_action = 2
    _action_idx = 2
    _action_distance = self.normal_punch_distance
    _action_dmg = self.normal_punch_dmg
    _delay = self.normal_punch_delay
    _groggy = self.normal_punch_groggy
    if self.groggy_count == 0 and self.delay == 0:
      # 액션이 가능한 상태인지를 체크한다.
      print(self.name,"가 보통 펀치를 날립니다")
      return _action_idx, _action_distance, _action_dmg, _delay, _groggy
    else:
      # 액션 수행 불가
      print(self.name,"가 보통 펀치를 날리려했으나 현재는 수행불가합니다")
      return -1, -1, -1, -1, -1

  def normal_punch_2(self): # Action Index 2
    self.current_action = 3
    _action_idx = 3
    _action_distance = self.normal_punch_2_distance
    _action_dmg = self.normal_punch_2_dmg
    _delay = self.normal_punch_2_delay
    _groggy = self.normal_punch_2_groggy
    if self.groggy_count == 0 and self.delay == 0:
      # 액션이 가능한 상태인지를 체크한다.
      print(self.name,"가 보통 펀치2를 날립니다")
      return _action_idx, _action_distance, _action_dmg, _delay, _groggy
    else:
      # 액션 수행 불가
      print(self.name,"가 보통 펀치2를 날리려했으나 현재는 수행불가합니다")
      return -1, -1, -1, -1, -1


  def normal_kick(self): # Action Index 3
    self.current_action = 4
    _action_idx = 4
    _action_distance = self.normal_kick_distance
    _action_dmg = self.normal_kick_dmg
    _delay = self.normal_kick_delay
    _groggy = self.normal_kick_groggy
    if self.groggy_count == 0 and self.delay == 0:
      # 액션이 가능한 상태인지를 체크한다.
      print(self.name,"가 보통 킥을 날립니다")
      return _action_idx, _action_distance, _action_dmg, _delay, _groggy
    else:
      # 액션 수행 불가
      print(self.name,"가 보통 킥을 날리려했으나 현재는 수행불가합니다")
      return -1, -1, -1, -1, -1

  def normal_kick_2(self): # Action Index 3
    self.current_action = 5
    _action_idx = 5
    _action_distance = self.normal_kick_2_distance
    _action_dmg = self.normal_kick_2_dmg
    _delay = self.normal_kick_2_delay
    _groggy = self.normal_kick_2_groggy 
    if self.groggy_count == 0 and self.delay == 0:
      # 액션이 가능한 상태인지를 체크한다.
      print(self.name,"가 보통 킥2을 날립니다")
      return _action_idx, _action_distance, _action_dmg, _delay, _groggy
    else:
      # 액션 수행 불가
      print(self.name,"가 보통 킥2을 날리려했으나 현재는 수행불가합니다")
      return -1, -1, -1, -1, -1

  def smash_punch(self): # Action Index 4
    # 사거리가 짧은 대신 데미지가 높고 상대방을 그로기로 몰아넣음
    self.current_action = 6
    _action_idx = 6
    _action_distance = self.smash_punch_distance
    _action_dmg = self.smash_punch_dmg
    _delay = self.smash_punch_delay
    _groggy = self.smash_punch_groggy 
    if self.groggy_count == 0and self.delay == 0:
      # 액션이 가능한 상태인지를 체크한다.
      print(self.name,"가 스매시 펀치를 날립니다")
      return _action_idx, _action_distance, _action_dmg, _delay, _groggy
    else:
      # 액션 수행 불가
      print(self.name,"가 스매시 펀치를 날리려했으나 현재는 수행불가합니다")
      return -1, -1, -1, -1, -1

  def smash_punch_2(self): # Action Index 4
    # 사거리가 짧은 대신 데미지가 높고 상대방을 그로기로 몰아넣음
    self.current_action = 7
    _action_idx = 7
    _action_distance = self.smash_punch_2_distance
    _action_dmg = self.smash_punch_2_dmg
    _delay = self.smash_punch_2_delay
    _groggy = self.smash_punch_2_groggy 
    if self.groggy_count == 0and self.delay == 0:
      # 액션이 가능한 상태인지를 체크한다.
      print(self.name,"가 스매시 펀치2를 날립니다")
      return _action_idx, _action_distance, _action_dmg, _delay, _groggy
    else:
      # 액션 수행 불가
      print(self.name,"가 스매시 펀치2를 날리려했으나 현재는 수행불가합니다")
      return -1, -1, -1, -1, -1


  def smash_kick(self): # Action Index 5
    # 사거리가 길고 데미지도 높지만 공격 이후 후딜이 존재함
    self.current_action = 8
    _action_idx = 8
    _action_distance = self.smash_kick_distance
    _action_dmg = self.smash_kick_dmg
    _delay = self.smash_kick_delay
    _groggy = self.smash_kick_groggy 
    if self.groggy_count == 0 and self.delay == 0:
      # 액션이 가능한 상태인지를 체크한다.
      print(self.name,"가 스매시 킥을 날립니다")
      return _action_idx, _action_distance, _action_dmg, _delay, _groggy
    else:
      # 액션 수행 불가
      print(self.name,"가 스매시 킥을 날리려했으나 현재는 수행불가합니다")
      return -1, -1, -1, -1, -1

  def smash_kick_2(self): # Action Index 5
    # 사거리가 길고 데미지도 높지만 공격 이후 후딜이 존재함
    self.current_action = 9
    _action_idx = 9
    _action_distance = self.smash_kick_2_distance
    _action_dmg = self.smash_kick_2_dmg
    _delay = self.smash_kick_2_delay
    _groggy = self.smash_kick_2_groggy  
    if self.groggy_count == 0 and self.delay == 0:
      # 액션이 가능한 상태인지를 체크한다.
      print(self.name,"가 스매시 킥2을 날립니다")
      return _action_idx, _action_distance, _action_dmg, _delay, _groggy
    else:
      # 액션 수행 불가
      print(self.name,"가 스매시 킥2을 날리려했으나 현재는 수행불가합니다")
      return -1, -1, -1, -1, -1

  def smash_kick_3(self): # Action Index 5
    # 사거리가 길고 데미지도 높지만 공격 이후 후딜이 존재함
    self.current_action = 10
    _action_idx = 10
    _action_distance = self.smash_kick_3_distance
    _action_dmg = self.smash_kick_3_dmg
    _delay = self.smash_kick_3_delay
    _groggy = self.smash_kick_3_groggy 
    if self.groggy_count == 0 and self.delay == 0:
      # 액션이 가능한 상태인지를 체크한다.
      print(self.name,"가 스매시 킥3을 날립니다")
      return _action_idx, _action_distance, _action_dmg, _delay, _groggy
    else:
      # 액션 수행 불가
      print(self.name,"가 스매시 킥3을 날리려했으나 현재는 수행불가합니다")
      return -1, -1, -1, -1, -1

  def guard(self): # Action Index 6
    # 상대방의 공격을 방어함
    self.current_action = 11
    _action_idx = 11
    _action_distance = 9999
    _action_dmg = 0
    _delay = 0
    _groggy = 0
    if self.groggy_count == 0 and self.delay == 0:
      # 액션이 가능한 상태인지를 체크한다.
      print(self.name,"가 가드를 올립니다")
      return _action_idx, _action_distance, _action_dmg, _delay, _groggy
    else:
      # 액션 수행 불가
      print(self.name,"가 가드를 올리려했으나 현재는 수행불가합니다")
      return -1, -1, -1, -1, -1

  def take_damage(self, dmg, _groggy_count):
    self.health = self.health - dmg
    self.groggy_count = _groggy_count
    print(self.name,"가 데미지를 입었습니다 현재 체력은 : ",self.health)
    pass

  def take_delay(self, _delay):
    self.delay = _delay
    pass

  def recover(self):
    if self.delay > 0:
      print(self.name,"의 남은 딜레이는 : ", self.delay)
      self.delay = self.delay - 1
    if self.groggy_count > 0:
      print(self.name,"의 남은 그로기 카운터는 : ", self.delay)
      self.groggy_count = self.groggy_count - 1
    pass
  
  def do_action(self, idx):
    if idx == 0:
      return self.move_left()
    elif idx == 1:
      return self.move_right()
    elif idx == 2:
      return self.normal_punch()
    elif idx == 3:
      return self.normal_punch_2()
    elif idx == 4:
      return self.normal_kick()
    elif idx == 5:
      return self.normal_kick_2()
    elif idx == 6 :
      return self.smash_punch()
    elif idx == 7 :
      return self.smash_punch_2()
    elif idx == 8 :
      return self.smash_kick()
    elif idx == 9 :
      return self.smash_kick_2()
    elif idx == 10 :
      return self.smash_kick_3()
    elif idx == 11 :
      return self.guard()

In [ ]:
is_rl_episode_end = False
agent_wins = 0 # 누적
agent_loses = 0 # 누적
game_draw = 0 # 누적
each_agent_wins = 0 # 개별
each_agent_loses = 0 # 개별
each_game_draw = 0 # 개별


class RLFighterEnvironment(py_environment.PyEnvironment):
    def __init__(self, _action, _observation, _state, _done):
        self._action_spec = _action
        self._observation_spec = _observation
        self._episode_ended = False
        self._reset_state = _state
        self._reset_done = _done
        self._state = _state
        self._done = _done

        self._penalty = -10 # When death agent
        self._praise = 1 # When kill an enemy
        self._log_idx = 0

        #JudgeMent
        self.agent_fighter = Fighter(100,500,"AgentFighter")
        self.enemy_fighter = Fighter(100,600,"EnemyFighter")
        self.wins = 0.0
        self.best_record = 9999
        self.num_turn = 0
  
    def action_spec(self):
        return self._action_spec
    
    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        self._state = self._reset_state
        self._done = self._reset_done
        self._episode_ended = False
        return ts.restart(self._reset_state)
    # custom function
    def send_action(self, action):
        # self.set_state_done_fromUE4()
        return 0.0
    
    # custom function
    def set_state_done_fromUE4(self):
        # self._done, self._state = 
        pass
    
    def get_distance(self):
      return abs(self.agent_fighter.current_loc - self.enemy_fighter.current_loc)


    def begin_action(self, actionIdx):
      # agent와 enemy의 액션 결정
      print("Agents Action : ", actionIdx)
      agent_action_idx, agent_action_distance, agent_action_dmg, agent_delay, agent_groggy = self.agent_fighter.do_action(actionIdx)
      # (적 테스트) 랜덤정수를 뽑는다
      enemy_action = random.randint(0,11)
      print("Enemys Random Action : ", enemy_action)
      enemy_action_idx, enemy_action_distance, enemy_action_dmg, enemy_delay, enemy_groggy = self.enemy_fighter.do_action(enemy_action)

      self.judge_result(agent_action_idx, agent_action_distance, agent_action_dmg, agent_delay, self.enemy_fighter, agent_groggy)
      self.judge_result(enemy_action_idx, enemy_action_distance, enemy_action_dmg, enemy_delay, self.agent_fighter, enemy_groggy)
      pass
    
    def judge_result(self, _action_idx, _action_distance, _action_dmg, _delay, _target, _groggy):
      # 두 파이터가 결정한 액션에 따라 판정결과를 리턴
      if _action_idx == -1: 
        # _action_idx = -1 -> 액션 수행 불가
        pass
      elif _action_idx == 0 or _action_idx == 1: 
        # _action_idx = 0,1 -> 이동판정
        pass
      elif _action_idx > 1 and _action_idx < 11: 
        # _action_idx = 2~5 -> 공격판정이므로 거리 데미지 딜레이 고려
        if _target.current_action == 11:
          print(_target.name,"이 가드상태입니다 이 공격은 무효화 됩니다.")
          # 가드판정이기때문에 패스
          pass
        else:
          if _action_distance > self.get_distance():
            _target.take_damage(_action_dmg, _groggy)
            _target.take_delay(_delay)
          else:
            pass
      elif _action_idx == 11: 
        # 
        pass
      pass

    def breaktime(self):
      # 두 파이터의 recover 함수를 호출하는것으로 딜레이감소, isactive 변수 활성화
      self.agent_fighter.recover()
      self.enemy_fighter.recover()
      pass

    def is_game_end(self):
      print("Action Result State : ", self._state)
      # 두 파이터의 체력을 비교하여 게임 결과를 선언
      if self.agent_fighter.health < 0:
        print("enemy win")
        return 0
      elif self.enemy_fighter.health < 0:
        print("agent win")
        return 1
      elif self.num_turn >= 100:
        return 3
      else:
        print("game continue red의 체력 : ",self.agent_fighter.health, "blue의 체력", self.enemy_fighter.health, "현재 턴수는 : ", self.num_turn, "최단 기록은 :", self.best_record)
        return 2

    def game_reset(self):
      self.agent_fighter.health = 100
      self.agent_fighter.current_loc = 500
      self.enemy_fighter.health = 100
      self.enemy_fighter.current_loc = 600

      self.agent_fighter.groggy_count = 0
      self.enemy_fighter.delay = 0
    
      self.num_turn = 0
      pass

    def renew_best_record(self):
      if self.num_turn <= self.best_record and self.num_turn != 0:
        self.best_record = self.num_turn
        return 5.0
      else:
        return 0.0

    def perfect_play_bonus(self):
      if self.agent_fighter.health == 100:
        return 3.0
      else:
        return 0.0

    def speed_game_bonus(self):
      game_speed = self.num_turn
      if game_speed <= 50:
        return 1.0
      elif game_speed <= 40:
        return 2.0
      elif game_speed <= 30:
        return 3.0
      elif game_speed <= 20:
        return 4.0
      elif game_speed <= 10:
        return 5.0
      elif game_speed <= 5:
        return 10.0
      else:
        return 0.0


    def _step(self, action):
        print("Action Value :", action)
        groggy_state = 0
        delay_state = 0
        if self.agent_fighter.groggy_count > 0:
          groggy_state = 1
        else:
          groggy_state = 0

        if self.agent_fighter.delay > 0:
          delay_state = 1
        else:
          delay_state = 0
        
        print("Before State is : ", self._state)
        self._state = np.array([self.agent_fighter.health, self.enemy_fighter.health, self.get_distance(), self.agent_fighter.current_action, self.enemy_fighter.current_action, groggy_state, delay_state], dtype=np.int32)
        print("After State is : ", self._state)
        #self._state = convert_tensor(np.array([self.agent_fighter.health, self.enemy_fighter.health, self.get_distance(), self.agent_fighter.current_action, self.enemy_fighter.current_action, self.agent_fighter.groggy_count, self.agent_fighter.delay], dtype=np.int64))
        # This function is custom function
        # Write down your RL algorithm
        
        # when episode is ended
        if self._episode_ended:
            return self.reset()
        
        # self.send_action(str(action))
        self.begin_action(action)
        
        global is_rl_episode_end
        global agent_wins
        global agent_loses
        global each_agent_wins 
        global each_agent_loses
        global each_game_draw
        global game_draw

        if self.is_game_end() == 2: # continue game
            print("Game Continue State Data : ", self._state)
            print("And Recent Winrate . . . : ", each_agent_wins, "Loses : ", each_agent_loses, "Draws : ", each_game_draw)
            self.num_turn = self.num_turn + 1
            self.breaktime()
            return ts.transition(
            self._state, reward= 0.0, discount=1.0)
        elif self.is_game_end() == 1: # agent win
            agent_wins = agent_wins + 1
            each_agent_wins = each_agent_wins + 1
            print("Agent Win Recent Wins : ", each_agent_wins, "Loses : ", each_agent_loses, "Draws : ", each_game_draw)
            best_record_bonus_reward = self.renew_best_record()
            perfect_bonums_reward = self.perfect_play_bonus()
            speed_bonus = self.speed_game_bonus()
            agent_reward = 1.0 + speed_bonus + best_record_bonus_reward + perfect_bonums_reward
            self.game_reset()
            is_rl_episode_end = True
            print("Give Reward : ", agent_reward)
            print("State Data : ", self._state)
            return ts.termination(self._state, agent_reward)
        elif self.is_game_end() == 0: # enemy win
            agent_loses = agent_loses + 1
            each_agent_loses = each_agent_loses + 1
            print("Enemy Win Recent Wins : ", each_agent_wins, "Loses : ", each_agent_loses)
            self.game_reset()
            agent_reward = -1.0
            is_rl_episode_end = True
            print("Give Reward : ", agent_reward)
            print("State Data : ", self._state)
            return ts.termination(self._state, agent_reward)
        elif self.is_game_end() == 3:
            print("State Data : ", self._state)
            print("Game Draw")
            each_game_draw = each_game_draw + 1
            game_draw = game_draw + 1
            print("Agent Win Recent Wins : ", each_agent_wins, "Loses : ", each_agent_loses)
            self.game_reset()
            is_rl_episode_end = True
            return ts.termination(self._state, 0.0)

In [ ]:
# 무엇을 인풋값으로 넣을것인가?
# 에이전트 체력, 적의 체력, 에이전트와 적의 거리, 에이전트의 액션, 적의 액션, 에이전트 그로기, 에이전트 딜레이


In [ ]:
_action_boundry_arr = array_spec.BoundedArraySpec(shape=(), dtype=np.int32, name='action', minimum=0, maximum=11)
_observation = array_spec.BoundedArraySpec(shape=(7,), dtype=np.int32, name='observation', 
                                                     minimum=[0, 0, 0, 0, 0, 0, 0], 
                                                     maximum=[100, 100, 1000, 11, 11, 1, 1])
_isDone = 0 # example
_stateArr = np.array([100, 100, 100, 0, 0, 0, 0], dtype=np.int32)
environment = RLFighterEnvironment(_action_boundry_arr, _observation, _stateArr, _isDone)
print('action_spec:', environment.action_spec())
print('time_step_spec.observation:', environment.time_step_spec().observation)
print('time_step_spec.step_type:', environment.time_step_spec().step_type)
print('time_step_spec.discount:', environment.time_step_spec().discount)
print('time_step_spec.reward:', environment.time_step_spec().reward)



action_spec: BoundedArraySpec(shape=(), dtype=dtype('int32'), name='action', minimum=0, maximum=11)
time_step_spec.observation: BoundedArraySpec(shape=(7,), dtype=dtype('int32'), name='observation', minimum=[0 0 0 0 0 0 0], maximum=[ 100  100 1000   11   11    1    1])
time_step_spec.step_type: ArraySpec(shape=(), dtype=dtype('int32'), name='step_type')
time_step_spec.discount: BoundedArraySpec(shape=(), dtype=dtype('float32'), name='discount', minimum=0.0, maximum=1.0)
time_step_spec.reward: ArraySpec(shape=(), dtype=dtype('float32'), name='reward')


In [ ]:
train_env = tf_py_environment.TFPyEnvironment(environment)
eval_env = tf_py_environment.TFPyEnvironment(environment)

num_iterations = 100000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"} 
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 1000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
gamma = 0.99 # for Categorical DQN
log_interval = 200  # @param {type:"integer"}

# for Categorical DQN
num_atoms = 51  # @param {type:"integer"}
min_q_value = -20  # custom
max_q_value = 20  # custom
n_step_update = 2  # @param {type:"integer"}


num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

fc_layer_params = (100, )

categorical_q_net = categorical_q_network.CategoricalQNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    num_atoms=num_atoms,
    fc_layer_params=fc_layer_params)

#q_net = q_network.QNetwork(
#    train_env.observation_spec(),
#    train_env.action_spec(),
#    fc_layer_params=fc_layer_params)

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

#train_step_counter = tf.Variable(0)
train_step_counter = tf.compat.v2.Variable(0)

agent = categorical_dqn_agent.CategoricalDqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    categorical_q_network=categorical_q_net,
    optimizer=optimizer,
    min_q_value=min_q_value,
    max_q_value=max_q_value,
    n_step_update=n_step_update,
    td_errors_loss_fn=common.element_wise_squared_loss,
    gamma=gamma,
    train_step_counter=train_step_counter)

#agent = dqn_agent.DqnAgent(
#    train_env.time_step_spec(),
#    train_env.action_spec(),
#    q_network=q_net,
#    optimizer=optimizer,
#    td_errors_loss_fn=common.element_wise_squared_loss,
#    train_step_counter=train_step_counter)

agent.initialize()

eval_policy = agent.policy
collect_policy = agent.collect_policy

random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

agent.collect_data_spec._fields



def collect_step(environment, policy, buffer):
    time_step = environment.current_time_step()
    print("Time Step : ", time_step)
    action_step = policy.action(time_step)
    print("Action Step : ", action_step)
    print("Selected Action Step : ", action_step.action)
    next_time_step = environment.step(action_step.action) 
    traj = trajectory.from_transition(time_step, action_step, next_time_step)

  # Add trajectory to the replay buffer
    buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps):
    for _ in range(steps):
        collect_step(env, policy, buffer)
    
    
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=n_step_update + 1).prefetch(3)

print("Dataset : ", dataset)

iterator = iter(dataset)

print("Iterator : ",iterator)


agent.train = common.function(agent.train)

agent.train_step_counter.assign(0)

import os
from tf_agents.policies import policy_saver

tempdir = "gdrive/MyDrive/RL_Fighter/models/temp/"
relearning_dir = "gdrive/MyDrive/RL_Fighter/models/modelCheckPoint/"
global_step = tf.compat.v1.train.get_or_create_global_step()

checkpoint_dir = os.path.join(tempdir, 'checkpoint')
checkpoint_dir = os.path.join(relearning_dir, 'checkpoint')
train_checkpointer = common.Checkpointer(
    ckpt_dir=checkpoint_dir,
    max_to_keep=1,
    agent=agent,
    policy=agent.policy,
    replay_buffer=replay_buffer,
    global_step=global_step
)

policy_dir = os.path.join(tempdir, 'policy')
tf_policy_saver = policy_saver.PolicySaver(agent.policy)

def saveModel():
    print("Save")
    train_checkpointer.save(global_step)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
Dataset :  <PrefetchDataset shapes: (Trajectory(step_type=(64, 3), observation=(64, 3, 7), action=(64, 3), policy_info=(), next_step_type=(64, 3), reward=(64, 3), discount=(64, 3)), BufferInfo(ids=(64, 3), probabilities=(64,))), types: (Trajectory(step_type=tf.int32, observation=tf.int32, action=tf.int32, policy_info=(), next_step_type=tf.int32, reward=tf.float32, discount=tf.float32), BufferInfo(ids=tf.int64, probabilities=tf.float32))>
Iterator :  <tensorflow.python.data.ops.iterator_ops.OwnedIterator object at 0x7f930b175908>


In [ ]:
# 정책이 던진 액션과
# csv에 명시된 액션을 비교한다
# 같으면 보상을주고
# 다르면 보상을 깎는다

def collect_step(environment, policy, buffer):
  time_step = environment.current_time_step()
  action_step = policy.action(time_step)
  next_time_step = environment.step(action_step.action)
  traj = trajectory.from_transition(time_step, action_step, next_time_step)
  buffer.add_batch(traj)

def calc_winrate(win, lose, total_game):
  print("Total Game : ", total_game)
  print("Total Win : ", win)
  print("Total Lose : ", lose)
  print("Total Draw : ", game_draw)
  if win + lose != 0:
    _winrate = (win / (win + lose)) * 100
  else:
    _winrate = 0 
  print("Win Rate : ", _winrate)

# 10000 에피소드 단위로 승률 변화 측정
# [Episode, win, lose, winrate, acc_winrate]



In [ ]:
evaluation = []
#winrate_arr = np.array([10000, 50, 50, 50, 50], dtype=np.int32)
def calc_evaluation(idx, _evaluation, _episode, _win, _lose, _winrate, _acc_winrate):
  winrate_arr = [_episode, _win, _lose, _winrate, _acc_winrate, game_draw]
  evaluation.append(winrate_arr)

#calc_evaluation(evaluation, 10000, 50, 50, 50, 50)
#calc_evaluation(evaluation, 20000, 50, 50, 50, 50)
#calc_evaluation(evaluation, 30000, 50, 50, 50, 50)
#calc_evaluation(evaluation, 40000, 50, 50, 50, 50)
#calc_evaluation(evaluation, 50000, 50, 50, 50, 50)

#print(evaluation[0])
#print(evaluation[1])
#print(evaluation[2])
#print(evaluation[3])
#print(evaluation[4])

In [ ]:
count = 0
# is_rl_episode_end = False
for _ in range(100):
  print("loop :", count)
  while is_rl_episode_end == False:
    collect_step(train_env, random_policy, replay_buffer)
  count = count + 1
  is_rl_episode_end = False

calc_winrate(agent_wins, agent_loses, 100)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Game Continue State Data :  [  9  62 350   5   7   0   0]
And Recent Winrate . . . :  34 Loses :  37 Draws :  1
Action Value : 6
Before State is :  [  9  62 350   5   7   0   0]
After State is :  [  9  62 350   6   5   0   0]
Agents Action :  6
AgentFighter 가 스매시 펀치를 날립니다
Enemys Random Action :  0
EnemyFighter 가 좌로 이동 수행 현재위치는 :  700
Action Result State :  [  9  62 350   6   5   0   0]
game continue red의 체력 :  9 blue의 체력 62 현재 턴수는 :  67 최단 기록은 : 4
Game Continue State Data :  [  9  62 350   6   5   0   0]
And Recent Winrate . . . :  34 Loses :  37 Draws :  1
Action Value : 1
Before State is :  [  9  62 350   6   5   0   0]
After State is :  [  9  62 300   6   0   0   0]
Agents Action :  1
AgentFighter 가 우로 이동 수행 현재위치는 :  450
Enemys Random Action :  3
EnemyFighter 가 보통 펀치2를 날립니다
Action Result State :  [  9  62 300   6   0   0   0]
game continue red의 체력 :  9 blue의 체력 62 현재 턴수는 :  68 최단 기록은 : 4
Game Continue State Data :  [  9  62 300   6   0   0   0]
An

In [ ]:
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
# avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
# returns = [avg_return]

evaluation_idx = 0

for j in range(100005):
  while is_rl_episode_end == False:
    print("Loop : ", j)
    collect_step(train_env, agent.collect_policy, replay_buffer)
  if j % 10000 == 0:
    saveModel()
    calc_evaluation(evaluation_idx, evaluation, j, each_agent_wins, each_agent_loses, calc_winrate(each_agent_wins, each_agent_loses, j), calc_winrate(agent_wins, agent_loses, j))
    each_agent_wins = 0
    each_agent_loses = 0
    each_game_draw = 0
  is_rl_episode_end = False

  experience, unused_info = next(iterator)
  train_loss = agent.train(experience)

print(evaluation)


#  step = agent.train_step_counter.numpy()

#  if step % log_interval == 0:
#    print('step = {0}: loss = {1}'.format(step, train_loss.loss))

  #if step % eval_interval == 0:
  #  avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
  #  print('step = {0}: Average Return = {1:.2f}'.format(step, avg_return))
  #  returns.append(avg_return)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs
Loop :  0
Action Value : 4
Before State is :  [100 100 100   0   0   0   0]
After State is :  [100 100 100   9   9   0   0]
Agents Action :  4
AgentFighter 가 보통 킥을 날립니다
Enemys Random Action :  1
EnemyFighter 가 우로 이동 수행 현재위치는 :  650
EnemyFighter 가 데미지를 입었습니다 현재 체력은 :  92
Action Result State :  [100 100 100   9   9   0   0]
game continue red의 체력 :  100 blue의 체력 92 현재 턴수는 :  0 최단 기록은 : 9999
Game Continue State Data :  [100 100 100   9   9   0   0]
And Recent Winrate . . . :  0 Loses :  0 Draws :  0
Loop :  0
Action Value : 6
Before State is :  [100 100 100   9   9   0   0]
After State is :  [100  92 150   4   1   0   0]
Agents Action :  6
AgentFighter 가 스매시 펀치를 날립니다
Enemys Random Action :  8
EnemyFighter 가 스매시 킥을 날립니다
EnemyFighter 가 데미지를 입었습니다 현재 체력은 :  77
AgentFighter 가 데미지를 입었습니다 현재 체력은 :  75
Action Result State :  [100  92 150   4   1   0   0]
game continue red의 체력 :  75 blue의 체력 77 현재 턴수는 :  1 최단 기록은 : 9999
Game Conti

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
And Recent Winrate . . . :  9221 Loses :  544 Draws :  211
EnemyFighter 의 남은 딜레이는 :  1
EnemyFighter 의 남은 그로기 카운터는 :  0
Loop :  99977
Action Value : 11
Before State is :  [100  20 100  10  10   0   0]
After State is :  [100  20 100  10  11   0   0]
Agents Action :  11
AgentFighter 가 가드를 올립니다
Enemys Random Action :  10
EnemyFighter 가 스매시 킥3을 날리려했으나 현재는 수행불가합니다
Action Result State :  [100  20 100  10  11   0   0]
game continue red의 체력 :  100 blue의 체력 20 현재 턴수는 :  6 최단 기록은 : 3
Game Continue State Data :  [100  20 100  10  11   0   0]
And Recent Winrate . . . :  9221 Loses :  544 Draws :  211
EnemyFighter 의 남은 그로기 카운터는 :  0
Loop :  99977
Action Value : 11
Before State is :  [100  20 100  10  11   0   0]
After State is :  [100  20 100  11  10   0   0]
Agents Action :  11
AgentFighter 가 가드를 올립니다
Enemys Random Action :  3
EnemyFighter 가 보통 펀치2를 날리려했으나 현재는 수행불가합니다
Action Result State :  [100  20 100  11  10   0   0]
game continue red의 체력 :  100 blue의 체력 20 현재

In [ ]:
import matplotlib
import matplotlib.pyplot as plt